In [8]:
import os
os.chdir(r"c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination")

import numpy as np
import pandas as pd
import torch
import transformers
from datasets import Dataset, load_dataset, load_from_disk, load_metric
from sklearn.metrics import mean_squared_error
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [2]:
df = pd.read_csv("data/cleaned_data/bulletins_w_labels_and_content.csv", dtype={'ID': object})  
df["Male share"] = round(df["Male"] / (df["Male"] +  df["Female"]), 3)
df = df[["Text", "Male share"]]
df

,Text,Male share
0,311 DIRECTOR Class Code: 9206 Open Date:...,0.608
1,ACCOUNTING CLERK Class Code: 1223 Open ...,0.238
2,AIRPORT MANAGER Class Code: 7260 Open D...,0.740
3,AIRPORT POLICE LIEUTENANT ...,0.809
4,AQUARIST Class Code: 2400 Open Date: 05...,0.615
...,...,...
172,WASTEWATER TREATMENT LABORATORY MANAGER Class...,0.600
173,WASTEWATER TREATMENT OPERATOR Class Code: ...,0.926
174,WATER MICROBIOLOGIST Class Code: 7857...,0.480
175,WATER UTILITY WORKER Class Code: 3912 Op...,0.979


In [3]:
dataset = Dataset.from_pandas(df, preserve_index=False) 
dataset = dataset.train_test_split(test_size=0.3) 
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'Male share'],
        num_rows: 123
    })
    test: Dataset({
        features: ['Text', 'Male share'],
        num_rows: 54
    })
})

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

100%|██████████| 1/1 [00:00<00:00,  8.79ba/s]


In [6]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

Downloading: 100%|██████████| 268M/268M [00:11<00:00, 22.8MB/s] 
c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination_sandbox\venv\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\britt\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint 

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

In [10]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["test"],
                  compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Male share, Text. If Male share, Text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination_sandbox\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 123
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24
  Number of trainable parameters = 66954241
  0%|          | 0/24 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
# save the model/tokenizer
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")
# load the model/tokenizer
from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [ ]:
#Using the model on new texts

trainer = Trainer(model=model)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) 

def pipeline_prediction(text):
    df= pd.DataFrame({'text':[text]})
    dataset = Dataset.from_pandas(df, preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred[0][0])